In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from datetime import datetime, timedelta
import pickle
import ipywidgets as widgets
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.rcParams['legend.fontsize'] = 10
plt.rcParams.update({'figure.autolayout': True})

## GPS_Stations Class
The `GPS_Stations` class is a wrapper for a data dictionary, `{Name: time series data}`. The time series data is an Nx4 ndarray, where N is the number of dates, from 2011/4/1 to 2016/12/31. The 4 columns correspond to date (`datetime` type), east, north and up components (all `float` type).

In [ ]:
class GPS_Stations():
    def __init__(self):
        self.data = {}
    def add(self, station_name, date, x, y, z):
        if station_name in self.data:
            # if the station is already in the data dictionary, append a new row of data [date,x,y,z]
            self.data.update( {station_name : np.append( self.data[station_name], np.array([date,x,y,z]).reshape(1,4), axis=0 ) } )
        else: # if first time adding this station, create a key:data pair.
            self.data.update( {station_name : np.array([date,x,y,z]).reshape(1,4)} )

## Loading Data
If a pickled data object exists, load and use it as an instance of `GPS_Stations` class named `Korea_GPS_Stations`. If not, i.e., `DATA_EXISTS==False`, read the data files retrived from the repository associated with (Kim et al., Tectonics, 2018) to create and populate an instance of `GPS_Stations` named `Korea_GPS_Stations`. 

If a pickled `GPS_Stations` data object exists, the first cell will just load it rather than read the coordinate files again. In this case, `DATA_EXISTS` is set to `True` so that the second cell can be skipped.

If such a pickled object does not exist (`DATA_EXISTS == False`), the second cell below should be run. Then, the newly created object is pickled for later use.

In [ ]:
DATA_EXISTS = False
try: 
    Korea_GPS_Stations = pickle.load( open( "Korea_GPS_Stations.p", "rb" ) )
    if Korea_GPS_Stations:
        DATA_EXISTS = True
        print("Data object exists")
except FileNotFoundError:
    print("Failed to load the pickled data. Move to the next cell.")

In [ ]:
if DATA_EXISTS == False:
    data_dir = "GPS Daily Coordinate data (2011 to 2016)/Coordinate"
    Korea_GPS_Stations = GPS_Stations()
    years = np.arange(2011,2017)
    #print(years)
    networks = np.array(['KASI', 'NGII'])
    #print(networks)
    for yr in years:
        for nt in networks:
            path = "./{0:s}/{1}/{2}".format(data_dir, yr, nt)
            #print(path)
            for f in os.listdir(path):
                fname = "{0}/{1}".format(path,f)
                fi = open(fname, 'r')
                lines = fi.readlines()
                date = np.datetime64(lines[2].split()[5]).astype(datetime)
                if date == datetime(2012,6,1): # exclude outlier
                    continue
                linecount = 7
                for line in lines[7:]:
                    linecount += 1
                    fields = line.split()
                    fieldnum = len(fields)
                    if fieldnum == 6:   # valid data without 'Name' field
                        name = fields[1]
                        x = float(fields[2])
                        y = float(fields[3])
                        z = float(fields[4])
                    elif fieldnum == 7: # valid data with all the fields filled
                        name = fields[1]
                        x = float(fields[3])
                        y = float(fields[4])
                        z = float(fields[5])
                    elif fieldnum == 5:   # invalid data with 0 0 0 coordinates
                        continue  # move on to the next line
                    elif fieldnum <= 1: # bottom blank lines
                        break     # stop processing this file
                    else:               # None of the above. Abort
                        print("Problem with line {0} in {1}".format(linecount, fname))
                        break     # stop processing this file
                    #print(fname, line)
                    Korea_GPS_Stations.add(name, date, x, y, z)
                fi.close()

    # Store the data object for quicker loading later
    with open("Korea_GPS_Stations.p", "wb") as handle:
        pickle.dump(Korea_GPS_Stations, handle, protocol=pickle.HIGHEST_PROTOCOL)
    DATA_EXISTS = True

In [ ]:
print( Korea_GPS_Stations.data['PUSN'][-1,1] - Korea_GPS_Stations.data['PUSN'][0,1] )
print( Korea_GPS_Stations.data['PUSN'][-1,0],Korea_GPS_Stations.data['PUSN'][0,0] )

In [ ]:
import os
for station in Korea_GPS_Stations.data.keys():
    cmd = "wget http://geodesy.unr.edu/gps_timeseries/txyz/IGS14/{0}.txyz2".format(station)
    os.sys(cmd)